In [ ]:
# 1. Self-attention by hand
# 2. Self-attention block in pytorch
# 3. GPT, piece-by-piece
# 4. GPU goes rrrr!

### Step 1: Self-attention by hand

In [1]:
import numpy as np

In [2]:
#  -- Write the scaled dot product self attention
  # 1. Compute queries, keys, and values
  # 2. Compute dot products
  # 3. Scale the dot products
  # 4. Apply softmax to calculate attentions
  # 5. Weight values by attentions
  # 6. Compute attention weighted features

In [3]:
# Choose values for the parameters
# Do not modify this code

X = np.array([[2,0,0,2],[0,1,0,0],[0,2,1,0],[0,0,1,1],[2,0,0,0],[1,0,1,1]], dtype=float)
W_Q = np.array([[1,1,0,0,0,0],[0,1,0,1,0,0],[0,0,1,0,1,1]], dtype=float)
W_K = np.array([[0,0,1,0,0,0],[0,1,0,0,0,0],[1,0,0,0,0,-1]], dtype=float)
W_V = np.array([[10,0,0,0,0,0],[0,0,0,10,0,0],[0,10,0,0,0,0]], dtype=float)

In [ ]:
# What does the first dimension of matrices Q and K correspond to?
# The Head size

In [4]:
# This is given for you. Do not modify this code.
def softmax_cols(data_in):
  # Exponentiate all of the values
  exp_values = np.exp(data_in)
  # Sum over columns
  denom = np.sum(exp_values, axis = 0)
  # Replicate denominator to N rows
  denom = np.matmul(np.ones((data_in.shape[0],1)), denom[np.newaxis,:])
  # Compute softmax
  softmax = exp_values / denom
  # return the answer
  return softmax

In [19]:
# compute the weighted attention matrix S
Q= W_Q@X
K= W_K@X
S = softmax_cols(np.transpose(K)@Q/np.sqrt(Q.shape[0]))


In [23]:
# compute the self-attention matrix A
V= W_V@X
A = V@S

In [25]:
# Sanity check. This should return True.
np.allclose(A, np.array([[10.30759701, 10.10551833, 15.03361159,  3.06082018],
       [ 2.83283874,  2.97334971,  4.13169018,  1.53041009],
       [ 4.59026201,  4.50027071,  2.10990693,  7.70438486]]))

True

### Step 2: Self-attention block in pytorch

In [26]:
import torch
import torch.nn as nn
from torch.functional import F

In [27]:
# do not modify this code

batch_size = 3 # B
block_size = 2 # T
n_embd = 3     # C

In [28]:
torch.set_printoptions(precision=8)

In [46]:
# Build a scaled self-attention head without masked attention and without dropout (i.e. just key, query and values)
# A matrix multiplication is implemented using the nn.Linear() operator with no bias.
class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.K = nn.Linear(n_embd, head_size, bias=False)
        self.Q = nn.Linear(n_embd, head_size, bias=False)
        self.V=nn.Linear(n_embd, head_size, bias=False)

    def forward (self, x):
        B, T, C = x.shape
        q=self.Q(x)
        k=self.K(x)
        v=self.V(x)
        print(q.shape, k.shape, v.shape)
        out = F.softmax(q@k.transpose(-2,-1)/np.sqrt(C), dim=-1)@v
        return out

In [47]:
# Unit test. Do not modify this code
torch.manual_seed(123) # do not remove this line
h = Head(2)
torch.manual_seed(123) # do not remove this line
x = torch.rand((batch_size, block_size, n_embd))
out = h(x)
out

torch.Size([3, 2, 2]) torch.Size([3, 2, 2]) torch.Size([3, 2, 2])


tensor([[[-0.46653441,  0.03306433],
         [-0.47224301,  0.04610372]],

        [[-0.38105938,  0.02397405],
         [-0.39453450,  0.02482041]],

        [[-0.29578221,  0.12158968],
         [-0.30042297,  0.12526260]]], grad_fn=<UnsafeViewBackward0>)

In [48]:
# Sanity check. This should return True.
torch.allclose(out, torch.tensor([[[-0.46653444,  0.03306433],
         [-0.47224304,  0.04610372]],
        [[-0.38105938,  0.02397406],
         [-0.39453450,  0.02482042]],
        [[-0.29578221,  0.12158968],
         [-0.30042297,  0.12526260]]]))

True

In [49]:
# Add weighted masked attention and dropout. Dropout comes after the softmax and before the multiplication with the value matrix.
# Copy the Head class from the previous exercise and expand upon it.

class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.K = nn.Linear(n_embd, head_size, bias=False)
        self.Q = nn.Linear(n_embd, head_size, bias=False)
        self.V=nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) # store a persistent buffer for the forward pass

    def forward (self, x):
        B, T, C = x.shape
        q=self.Q(x)
        k=self.K(x)
        v=self.V(x)
        wei=q@k.transpose(-2,-1)/np.sqrt(C)
        wei=wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1)
        out = wei@v
        return out

In [50]:
# Unit test. Do not modify this code
torch.manual_seed(123) # do not remove this line
h = Head(2)
torch.manual_seed(123) # do not remove this line
x = torch.rand((batch_size, block_size, n_embd))
out = h(x)
out

tensor([[[-0.37939817, -0.16596894],
         [-0.47224301,  0.04610372]],

        [[-0.14184165,  0.00894911],
         [-0.39453450,  0.02482041]],

        [[-0.17301908,  0.02442870],
         [-0.30042297,  0.12526260]]], grad_fn=<UnsafeViewBackward0>)

In [51]:
# Sanity check. This should return True.
torch.allclose(out, torch.tensor([[[-0.37939820, -0.16596894],
         [-0.47224304,  0.04610372]],
        [[-0.14184165,  0.00894911],
         [-0.39453450,  0.02482042]],
        [[-0.17301908,  0.02442869],
         [-0.30042297,  0.12526260]]]))


True

In [56]:
# A multi-head attention module contains a list of heads and a linear projection layer.
# The heads are applied to the input and then concatenated along the last dimension, then
# the linear layer is applied. Look at the unit test below to determine the dimensions of
# the linear layer.

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.linear = nn.Linear(num_heads*head_size, n_embd)
        

    def forward (self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.linear(out)
        return out

In [57]:
# do not modify
num_heads = 3
head_size = 2
n_embd = 6

In [58]:
# Unit test. Do not modify this code
torch.manual_seed(123) # do not remove this line
sa = MultiHeadAttention(num_heads=3, head_size=head_size)
torch.manual_seed(123) # do not remove this line
x = torch.rand((batch_size, block_size, n_embd))
out = sa(x)

In [59]:
# Sanity check. This should return True.
torch.allclose(out, torch.tensor([[[-0.03730504, -0.07006130, -0.27096999,  0.13144857, -0.45049590,
          -0.33217290],
         [-0.06794342, -0.04509801, -0.34738648,  0.15599491, -0.45456851,
          -0.33087400]],
        [[-0.08914752, -0.03846309, -0.36569631,  0.09802882, -0.39963537,
          -0.29225215],
         [-0.04709741,  0.01406255, -0.25430590,  0.08396727, -0.41786054,
          -0.30781299]],
        [[ 0.15234883, -0.08591781, -0.10099770,  0.19886394, -0.49236685,
          -0.43605998],
         [ 0.15425430, -0.01792544, -0.00511202,  0.14046597, -0.48078871,
          -0.40730378]]]))


True

In [67]:
# Add a classical feedforward module: linear -> ReLU -> linear
# The hidden dimension is four times bigger than the input dimension (see Section 3.3 of Attention is All You Need)
#
class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """
    def __init__(self, n_embd):
        super().__init__() 
        self.model=nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd)
        )
        

    def forward(self, x):
        out=self.model(x)
        return out

In [68]:
# Unit test. Do not modify this code
torch.manual_seed(123) # do not remove this line
ff = FeedForward(n_embd)
torch.manual_seed(123) # do not remove this line
x = torch.rand((3,n_embd))
out = ff(x)
out

tensor([[-0.58034134,  0.04641047, -0.10707696,  0.21581653, -0.30361828,
         -0.07352638],
        [-0.48917407,  0.07879594, -0.15972012,  0.17862342, -0.37070659,
         -0.07852858],
        [-0.48530388,  0.09604470, -0.06524839,  0.16611034, -0.35499069,
         -0.08964306]], grad_fn=<AddmmBackward0>)

In [69]:
# Sanity check. This should return True.
torch.allclose(out, torch.tensor([[-0.58034140,  0.04641046, -0.10707694,  0.21581653, -0.30361831,
         -0.07352637],
        [-0.48917407,  0.07879593, -0.15972012,  0.17862344, -0.37070659,
         -0.07852858],
        [-0.48530388,  0.09604470, -0.06524836,  0.16611034, -0.35499069,
         -0.08964306]]))

True

In [70]:
# Build a self-attention block
#
#   in -----> LayerNorm -------> multi-head attention -- + ----> LayerNorm -----> FeedForward --- + -----> out
#         |                                              |   |                                    |
#          ----------------------------------------------     ------------------------------------                       
#
# This architecture is slightly different from Attention is All You Need (or the UDL textbook):
# the layer norm comes before (not after) the attention or feed-forward
#
class Block(nn.Module):

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.mH=MultiHeadAttention(n_head,head_size)
        self.layer1=nn.LayerNorm(n_embd)
        self.ff=FeedForward(n_embd)
        self.layer2=nn.LayerNorm(n_embd)

        

    def forward(self, x):
        y = self.mH(self.layer1(x)) + x
        out=self.ff(self.layer2(y)) + y
        return out

In [74]:
# Unit test. Do not modify this code
torch.manual_seed(123) # do not remove this line
bk = Block(n_embd, num_heads)
torch.manual_seed(123) # do not remove this line
x = torch.rand((batch_size,block_size,n_embd))
out = bk(x)
out

tensor([[[-0.05278995, -0.10863629, -0.09458941,  0.97590691, -0.55101192,
           0.57085061],
         [-0.17928872, -0.44799614, -0.26547033,  1.11293721, -0.34837404,
           0.40728986]],

        [[-0.41515028, -0.30126402, -0.11399305,  0.64651299, -0.51579154,
           0.57017863],
         [-0.02734706,  0.08873296,  0.65776676,  0.70304358,  0.05667022,
           0.70008963]],

        [[ 0.52881181,  0.34458160,  0.31130394,  1.11564195,  0.37998515,
          -0.02971911],
         [ 1.39032197,  0.58906519,  0.97761846,  0.38604790,  0.63349819,
           0.50254494]]], grad_fn=<AddBackward0>)

In [75]:
# Sanity check. This should return True.
torch.allclose(out, torch.tensor([[[-0.05278997, -0.10863629, -0.09458938,  0.97590691, -0.55101192,
           0.57085067],
         [-0.17928867, -0.44799608, -0.26547045,  1.11293721, -0.34837404,
           0.40728986]],
        [[-0.41515028, -0.30126408, -0.11399293,  0.64651299, -0.51579159,
           0.57017863],
         [-0.02734703,  0.08873296,  0.65776664,  0.70304352,  0.05667025,
           0.70008957]],
        [[ 0.52881187,  0.34458166,  0.31130391,  1.11564195,  0.37998506,
          -0.02971917],
         [ 1.39032197,  0.58906519,  0.97761846,  0.38604784,  0.63349819,
           0.50254500]]]))

True

In [ ]:
## Step 3: Build a mini GPT
#
# - Start from the gpt-problem.py file
# - Add your Head, MultiHeadAttention, FeedForward and Block classes
# - Fill in the GPT class (__init__ and forward methods)
# - Train the network on CPU
# - Train the network on GPU

# For __init__, the GPT model parameters are:
#   - a token embedding table
#   - a positional embedding table
#   - a sequence of Blocks
#   - a layer norm
#   - a linear layer
#
# For forward(), the model consists in:
#   - applying the token embedding table and positional embedding table to the input tensor
#   - adding the two together
#   - applying the blocks, layer norm and linear layer (in that order)
#
# The code comes from hyperparameters that should work well on GPU.  On CPU, you 
# will need to reduce the model size significantly.
#
# In pytorch, an learnable embedding table is implemented with nn.Embedding(...)
#
# The token embedding table learns an embedding for each item of the vocabulary. The 
# positional embedding table does not depend on the input and learns an embedding
# for each position in the context.